In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [ ]:
import numpy as np

# Get Plane

In [ ]:
from Planes.hermes import  hermes
airplane = hermes("hermes")
airplane.visualize()

In [ ]:
airplane.span

# Import Environment and Set State

In [ ]:
from ICARUS.environment.definition import EARTH_ISA
from ICARUS.flight_dynamics.state import State

u_freestream = 20
unstick = State(
    name= "unstick",
    airplane= airplane,
    environment= EARTH_ISA,
    u_freestream= u_freestream
)

# Get Solver

In [ ]:
from ICARUS.computation.solvers.Icarus_LSPT.wing_lspt import LSPT
lspt = LSPT()

print(lspt)


## AoA Run

In [ ]:
analysis: str = lspt.get_analyses_names()[0]
print(f"Selecting Analysis: {analysis}")
lspt.select_analysis(analysis)

In [ ]:
options = lspt.get_analysis_options(verbose=True)
solver_parameters = lspt.get_solver_parameters(verbose=True)

In [ ]:
from ICARUS.aerodynamics.wing_lspt import LSPT_Plane
plane_LSPT = LSPT_Plane(plane = airplane)
plane_LSPT.alpha = 5

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax: Axes3D = fig.add_subplot(111, projection='3d') # type: ignore
plane_LSPT.plot_panels(ax = ax)

In [ ]:
plane_LSPT.surface_dict['rudder']

In [ ]:
plane_LSPT.aseq(
    state = unstick,
    angles = np.linspace(-10, 10, 20),
)

In [ ]:
plane_LSPT.plot_surface_gamma_distribution()

In [ ]:
# Plot the panels and lift
plane_LSPT.plot_L_pan()
plane_LSPT.plot_D_pan()

In [ ]:
plane_LSPT.L_pan

In [ ]:
AoAmin = -6
AoAmax = 8
NoAoA = (AoAmax - AoAmin) + 1
angles = np.linspace(AoAmin, AoAmax, NoAoA, dtype= float)
# airplane.define_dynamic_pressure(u_freestream, EARTH.air_density)

options.plane         = airplane
options.state         = unstick
options.solver2D      = 'Xfoil'
# options.maxiter       = 100
# options.timestep      = 0.001
options.angles        = angles

solver_parameters.Use_Grid = 0

lspt.define_analysis(options, solver_parameters)
lspt.print_analysis_options()
lspt.execute(parallel = False)
polars = lspt.get_results()
airplane.save()

# Dynamics

### Define and Trim Plane

In [ ]:
from ICARUS.flight_dynamics.state import State
if isinstance(polars,int):
    raise Exception("No Polars")
unstick = State("Unstick",airplane,environment= EARTH_ISA, u_freestream= u_freestream)
unstick.add_polar(polars)

### Pertrubations

In [ ]:
unstick.add_all_pertrubations("Central")
unstick.get_pertrub()

In [ ]:
analysis = lspt.get_analyses_names(verbose=True)[1] # ANGLES PARALLEL
print(f"Selecting Analysis: {analysis}")
lspt.select_analysis(analysis)
options = lspt.get_analysis_options(verbose=True)

In [ ]:
options.plane         = airplane
options.state         = unstick
options.solver2D      = 'XFLR'
options.maxiter       = 200
options.timestep      = 5e-3

lspt.print_analysis_options()
lspt.define_analysis(options, solver_parameters)
lspt.execute(parallel=True)
lspt.get_results()